# Reconnaissance d'entités nommées avec SpaCy

La documentation est accessible ici: https://spacy.io/api

## Imports

In [ ]:
from collections import defaultdict
import time 
import spacy
!python -m spacy download fr_core_news_md

In [2]:
nlp = spacy.load('fr_core_news_md')

In [3]:
file_path = "../../data/tmp/1950_clean.txt"

## Appliquer la reconnaissance d'entités nommées sur notre corpus

In [ ]:
# Charger le texte
n=1000000
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()[:n] 
    print(f"Texte chargé. Premiers 500 caractères :\n{text[:500]}...")
except FileNotFoundError:
    print(f"Erreur : Le fichier '{file_path}' n'a pas été trouvé. Veuillez vérifier le chemin.")
    exit() 

In [ ]:
%%time
# Traiter le texte

try:
    nlp = spacy.load("fr_core_news_md") # Charger le modèle spaCy
    print("\nModèle spaCy chargé avec succès.")
except OSError:
    print("\nErreur : Le modèle spaCy 'fr_core_news_md' n'est pas installé.")
    
    exit()

# Traitement du texte
start_time = time.time() 
doc = nlp(text)
end_time = time.time()
print(f"Texte traité en {end_time - start_time:.2f} secondes.")

### Compter les entités "Personne" 

In [ ]:
people = defaultdict(int)

for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1 

print(f"\nNombre d'entités 'Personne' uniques trouvées : {len(people)}")
sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
print("\n--- Top 50 des personnes les plus fréquentes ---")

# Afficher les 50 premières personnes et leurs fréquences
for person, freq in sorted_people[:50]:
    print(f'"{person}" apparaît {freq} fois dans le corpus')

Exercice: essayez de lister les lieux (LOC) et les organisations (ORG) les plus mentionnées dans le corpus

In [ ]:
categorized_entities_counts = defaultdict(lambda: defaultdict(int))
total_entities_found = 0

for ent in doc.ents:
    categorized_entities_counts[ent.label_][ent.text] += 1
    total_entities_found += 1

print("\n Résultat de l'extraction des entités nommées ")
if total_entities_found == 0:
    print("Le texte ne contient aucune entité nommée détectée par le modèle.")
else:
    print(f"Un total de {total_entities_found} entités (y compris les doublons) ont été détectées.")
    print("\n Entités nommées qui apparaissent 2 fois ou plus ")

    found_any_recurring = False

    for label in sorted(categorized_entities_counts.keys()):
        entities_in_category = [
            (entity_text, count)
            for entity_text, count in categorized_entities_counts[label].items()
            if count >= 2
        ]

        if entities_in_category:
            found_any_recurring = True
            entities_in_category_sorted = sorted(entities_in_category, key=lambda item: item[1], reverse=True)

            print(f"\n Catégorie : {label} ({len(entities_in_category_sorted)} entité(s) récurrente(s))")
            for entity_text, count in entities_in_category_sorted:
                print(f"- {entity_text} : apparaît {count} fois")

    if not found_any_recurring:
        print("\n Aucune entité nommée n'apparaît 2 fois ou plus dans le texte.")
